In [1]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np

# التحقق من توفر CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Running on device: {device}")

# تحميل نموذج YOLO إلى الجهاز المناسب
model = YOLO("runs\\detect\\train3\\weights\\best.pt").to(device)

# فتح الكاميرا
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# تحديد منطقة الدخول (ROI)
x1, y1, x2, y2 = 200, 100, 500, 400 

# عداد الدخول فقط
entered = 0

# مجموعة IDs تم عدّها
counted_ids = set()

# دالة للتحقق إن كانت النقطة داخل المنطقة
def is_inside_roi(cx, cy):
    return x1 < cx < x2 and y1 < cy < y2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # استخدام YOLO مع التتبع
    results = model.track(frame, persist=True, tracker="botsort.yaml", verbose=False)

    # إذا تم الكشف عن كائنات
    if results[0].boxes.id is not None:
        ids = results[0].boxes.id.cpu().numpy()
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()

        # رسم المنطقة
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Entered: {entered}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        for i, box in enumerate(boxes):
            class_id = int(classes[i])
            if class_id != 0:  # فقط الأشخاص
                continue

            x1b, y1b, x2b, y2b = box[:4]
            cx, cy = int((x1b + x2b) / 2), int((y1b + y2b) / 2)
            obj_id = int(ids[i])

            # رسم مركز الكائن ومعرّفه
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, str(obj_id), (cx, cy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            # عدّ الداخلين فقط (مرة واحدة لكل ID)
            if is_inside_roi(cx, cy) and obj_id not in counted_ids:
                entered += 1
                counted_ids.add(obj_id)

    # عرض النتيجة
    cv2.imshow("عدّ الدخول باستخدام YOLO (CUDA)", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


🚀 Running on device: cpu
